In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- 1. DATASET POUR LE DÉBRUITAGE (MNIST) ---
# Utilisation de MNIST pour un entraînement ultra-rapide
(x_train_mnist, _), (x_test_mnist, _) = tf.keras.datasets.mnist.load_data()
x_train_mnist = x_train_mnist.astype('float32') / 255.
x_test_mnist = x_test_mnist.astype('float32') / 255.
# On ajoute une dimension pour le canal (28, 28, 1)
x_train_mnist = np.expand_dims(x_train_mnist, -1)
x_test_mnist = np.expand_dims(x_test_mnist, -1)


# --- 2. DATASET POUR LA COLORISATION (FLOWERS 102) ---
# Chargement via TFDS (Plus lourd, nécessite une connexion internet)
# Si Flowers est trop lourd, vous pouvez swapper avec 'cifar10'
ds_flowers, info_flowers = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
# Note : ds_flowers est un dictionnaire contenant 'train', 'test' et 'validation'


# --- 3. DATASET POUR LA SUPER-RÉSOLUTION (STL-10 ou CIFAR-10) ---
# On utilise CIFAR-10 ici pour la compatibilité, STL-10 est aussi possible via tfds
(x_train_sr, _), (x_test_sr, _) = tf.keras.datasets.cifar10.load_data()
x_train_sr = x_train_sr.astype('float32') / 255.
x_test_sr = x_test_sr.astype('float32') / 255.


# --- 4. DATASET POUR LA DÉTECTION D'ANOMALIES (CREDIT CARD FRAUD) ---
# Lien de téléchargement : https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
# Assurez-vous que le fichier 'creditcard.csv' est dans votre dossier racine
try:
    df_fraud = pd.read_csv('creditcard.csv')
    
    # 1. On sépare les normales des fraudes
    df_normal = df_fraud[df_fraud['Class'] == 0].drop(['Class', 'Time'], axis=1)
    df_anomaly = df_fraud[df_fraud['Class'] == 1].drop(['Class', 'Time'], axis=1)
    
    # 2. Préparation de l'entraînement (Uniquement sur des transactions normales)
    # On prend 50 000 normales pour l'entraînement
    x_normal_train, x_normal_test = train_test_split(
        df_normal.sample(n=50000, random_state=42), 
        test_size=0.2, 
        random_state=42
    )
    
    # 3. Normalisation (On calibre le scaler sur le train normal UNIQUEMENT)
    scaler = StandardScaler()
    x_train_anomaly = scaler.fit_transform(x_normal_train)
    x_test_normal = scaler.transform(x_normal_test)
    
    # 4. Préparation du set de FRAUDES pour le test final
    # On normalise les fraudes avec le même scaler
    x_test_fraud = scaler.transform(df_anomaly)
    
    print(f"✅ Dataset chargé.")
    print(f"🏠 Transactions normales pour l'entraînement : {x_train_anomaly.shape[0]}")
    print(f"🚨 Transactions frauduleuses pour le test final : {x_test_fraud.shape[0]}")

except FileNotFoundError:
    print("⚠️ Fichier 'creditcard.csv' non trouvé.")

# --- RÉSUMÉ DES DIMENSIONS ---
print(f"\n📊 Denoising (MNIST): {x_train_mnist.shape}")
print(f"📊 Super-Res (CIFAR): {x_train_sr.shape}")
print(f"📊 Anomaly (Fraud): {x_train_anomaly.shape if 'x_train_anomaly' in locals() else 'N/A'}")
